In [2]:
import pandas as pd
import numpy as np
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

In [31]:
def checker(wrong_options,correct_options, cf_dict):
    names_array=[]
    ratio_array=[]
    score_array = []
    i = 0
    for wrong_option in wrong_options:
        if wrong_option in correct_options:
            names_array.append(wrong_option)
            ratio_array.append('100')
            score_array.append(cf_dict[wrong_option])
        else:   
            x=process.extractOne(wrong_option,correct_options,scorer=fuzz.token_set_ratio)
            names_array.append(x[0])
            ratio_array.append(x[1])
            score_array.append(cf_dict[x[0]])
        i += 1
        if i % 100 == 0:
            print(i)
    return names_array, ratio_array, score_array

In [11]:
con_df = pd.read_csv("aggregated_contribution_by_candidate_party_year_district.csv", dtype={'CANDIDATE':'string'})
dime_df = pd.read_csv('dime_recipients_1979_2014.csv',dtype={'name':'string'})

In [9]:
#Take unique values of dfs
dime_df['name'] = dime_df['name'].fillna('####').str.lower()
dime_df['recipient.cfscore'] = dime_df['recipient.cfscore'].fillna('####')
dime_df = dime_df.drop_duplicates(subset=['name'], keep='last')
con_df['CANDIDATE'] = con_df['CANDIDATE'].fillna('####').str.lower().str.replace(' senator', '', regex=False)
con_df = con_df.drop_duplicates(subset=['CANDIDATE'], keep='first')

In [29]:
str_options = dime_df['name'].tolist()
str2match = con_df['CANDIDATE'].tolist()
cf_scores = dict(zip(dime_df['name'], dime_df['recipient.cfscore']))

In [35]:
con_df.shape

(13211, 6)

In [32]:
matched_name, match_ratio, match_cf = checker(str2match, str_options, cf_scores)

100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500
9600
9700
9800
9900
10000
10100
10200
10300
10400
10500
10600
10700
10800
10900
11000
11100
11200
11300
11400
11500
11600
11700
11800
11900
12000
12100
12200
12300
12400
12500
12600
12700
12800
12900
13000
13100
13200


In [36]:
matched_df = pd.DataFrame()
matched_df['contribution_name'] = pd.Series(str2match)
matched_df['dime_name'] = pd.Series(matched_name)
matched_df['ratio'] = pd.Series(match_ratio)
matched_df['cfscore'] = pd.Series(match_cf)

In [37]:
matched_df

,contribution_name,dime_name,ratio,cfscore
0,"abercrombie, neil","abercrombie, neil",100,-0.862
1,"abraham, spencer","abraham, edmond spencer",100,0.820
2,"acevedo-vila, anibal","acevedo vila, anibal",100,-0.678
3,"ackerman, gary l","ackerman, gary l",100,-0.602
4,"aderholt, robert brown","aderholt, robert brown",100,0.830
...,...,...,...,...
13206,"witzke, lauren elena","parent, elena",71,-0.542
13207,"wright, christopher robert","wright, robert",100,1.202
13208,"wyman, robert mark (bob)","harman, robert (bob)",84,0.261
13209,"yacus, george michael","liffrig, michael george",82,1.615


In [39]:
matched_df.to_csv("matched_scores.csv")

In [5]:
matched_df = pd.read_csv("matched_scores.csv")

In [6]:
#Filter ratios to >= .95, make dict of contribution name:cfscore, add scores to contribution data, then aggregate by district
pruned_df = matched_df[matched_df['ratio'] >= 95]

In [7]:
#69% of unique candidates matched to scores
pruned_df

,Unnamed: 0,contribution_name,dime_name,ratio,cfscore
0,0,"abercrombie, neil","abercrombie, neil",100,-0.862
1,1,"abraham, spencer","abraham, edmond spencer",100,0.820
2,2,"acevedo-vila, anibal","acevedo vila, anibal",100,-0.678
3,3,"ackerman, gary l","ackerman, gary l",100,-0.602
4,4,"aderholt, robert brown","aderholt, robert brown",100,0.830
...,...,...,...,...,...
13198,13198,"weber, robert marion","weber, robert",100,-1.028
13203,13203,"wilkinson, john","wilkinson, john k",100,1.209
13204,13204,"williams, william henry","williams, william",100,0.071
13207,13207,"wright, christopher robert","wright, robert",100,1.202


In [12]:
#Make the dict and get con_df ready for mapping
candidate_score = dict(zip(pruned_df['contribution_name'], pruned_df['cfscore']))
con_df['CANDIDATE'] = con_df['CANDIDATE'].fillna('####').str.lower().str.replace(' senator', '', regex=False)

In [17]:
#Map scores to con_df
con_df['CFSCORE'] = con_df['CANDIDATE'].map(candidate_score)

In [18]:
con_df

,CANDIDATE,YEAR,PTY_AFFILIATION,DISTRICT,COUNT,SUM,CFSCORE
0,"abercrombie, neil",1999,DEM,ARIZONA,1,500,-0.862
1,"abercrombie, neil",1999,DEM,CALIFORNIA CENTRAL,3,3000,-0.862
2,"abercrombie, neil",1999,DEM,CALIFORNIA NORTHERN,11,16250,-0.862
3,"abercrombie, neil",1999,DEM,CALIFORNIA SOUTHERN,3,1250,-0.862
4,"abercrombie, neil",1999,DEM,DISTRICT OF COLUMBIA,8,4050,-0.862
...,...,...,...,...,...,...,...
533889,"zunker, tricia",2020,DEM,TEXAS WESTERN,1,100,NaN
533890,"zunker, tricia",2020,DEM,WASHINGTON WESTERN,4,5950,NaN
533891,"zunker, tricia",2020,DEM,WEST VIRGINIA SOUTHERN,1,250,NaN
533892,"zunker, tricia",2020,DEM,WISCONSIN EASTERN,76,37781,NaN


In [24]:
#Lose about 18.5% from aggregated contributions
pruned_con = con_df[con_df['CFSCORE'].notna()]

In [25]:
pruned_con.to_csv("aggregated_contribution_cfscore.csv")

In [28]:
#Find weighted average scores across districts
districts = con_df['DISTRICT'].unique().tolist()

In [32]:
con_df[con_df['DISTRICT'] == 'ARIZONA']

,CANDIDATE,YEAR,PTY_AFFILIATION,DISTRICT,COUNT,SUM,CFSCORE
0,"abercrombie, neil",1999,DEM,ARIZONA,1,500,-0.862
18,"abraham, spencer",1999,REP,ARIZONA,13,14900,0.820
150,"alexander, andrew lamar",1999,REP,ARIZONA,5,3250,0.916
227,"allen, george",1999,REP,ARIZONA,38,25250,0.964
307,"amores, jon",1999,DEM,ARIZONA,1,250,-0.070
...,...,...,...,...,...,...,...
533675,"young, david",2020,REP,ARIZONA,2,850,-0.483
533706,"young, tammy",2020,DEM,ARIZONA,1,500,NaN
533729,"young, todd christopher",2020,REP,ARIZONA,2,800,1.551
533770,"yu, eugene chin mr.",2020,REP,ARIZONA,1,250,1.375


In [43]:
temp = con_df[con_df['DISTRICT'] == 'DELAWARE']
total = temp['SUM'].sum()
mult = temp['SUM'] * temp['CFSCORE']
sum_mult = mult.sum()
final = sum_mult / total
sum_dict['DELAWARE'] = final

In [46]:
#Make dicts to populate df later - going to use two different versions of the aggregated scores
count_dict = {}
sum_dict = {}
district_count_dict = {}


In [47]:
#populate sum dict
for district in districts:
    temp = con_df[con_df['DISTRICT'] == district]
    total = temp['SUM'].sum()
    mult = temp['SUM'] * temp['CFSCORE']
    sum_mult = mult.sum()
    final = sum_mult / total
    sum_dict[district] = final
    
#populate count_dict
for district in districts:
    temp = con_df[con_df['DISTRICT'] == district]
    total = temp['COUNT'].sum()
    mult = temp['COUNT'] * temp['CFSCORE']
    sum_mult = mult.sum()
    final = sum_mult / total
    district_count_dict[district] = total
    count_dict[district] = final

In [64]:
#Create district partisan df
district_partisanship_df = pd.DataFrame.from_dict(count_dict, orient='index')
district_partisanship_df.reset_index(inplace=True)
district_partisanship_df = district_partisanship_df.rename(columns = {0 : 'count_score', 'index': 'district'})

In [70]:
district_partisanship_df["sum_score"] = district_partisanship_df['district'].map(sum_dict)
district_partisanship_df["total_count"] = district_partisanship_df['district'].map(district_count_dict)

In [71]:
district_partisanship_df

,district,count_score,sum_score,total_count
0,ARIZONA,0.052798,0.164299,229078
1,CALIFORNIA CENTRAL,-0.380242,-0.298018,742059
2,CALIFORNIA NORTHERN,-0.603477,-0.344415,656293
3,CALIFORNIA SOUTHERN,-0.242474,-0.099297,130838
4,DISTRICT OF COLUMBIA,-0.405676,-0.406253,350789
...,...,...,...,...
88,ILLINOIS SOUTHERN,0.013458,0.018961,29401
89,MONTANA,-0.197312,-0.036750,59473
90,LOUISIANA MIDDLE,0.355271,0.406590,28226
91,GUAM,0.064671,0.132586,2217


In [72]:
district_partisanship_df.to_csv('district_partisanship.csv')